###Travel Data Analysis using Apache Spark on Databricks Community Edition
In this notebook, We will be demonstrating various concepts of Apache Spark like actions and transformation with the help of an example. WE will be executing the examples using various API's present in Apache Spark such as RDD's.


###Dataset
1. <a href="https://raw.githubusercontent.com/kavyaprasad/data/master/TravelData.csv"><b>TravelData.csv</b></a> - This file contails data about Travel. The various attributes present in the dataset are
  1. City Pair: The origin-destination pair of travel (STRING)
  2. From Location: The origin of travel (STRING)
  3. To Location: The destination of travel (STRING)
  4. Product Type: This attribute tells us about the mode of travel and the various combinations which the traveller opted. Here 1 represents Air, 2 represents car,    represents Air+Car, 4 represents Hotel, 5 represents Air+Hotel, 6 represents Hotel+Car, 7 represents Air+Hotel+Car) (INTEGER)
  5. Adults Traveling: Number of adults travelling (INTEGER)
  6. Seniors Traveling: Number of seniors travelling (INTEGER)
  7. Children Traveling:  Number of children travelling (INTEGER)
  8. Youth Traveling:  Number of youth travelling (INTEGER)
  9. Infant Traveling:  Number of infants travelling (INTEGER)
  10. Date of Travel: The date of travel(STRING)
  11. Time of Travel: The time of travel(STRING)
  12. Date of Return: The date of return (STRING)
  13. Time of Return: The time of return (STRING)
  14. Price of booking: The price of booking hotel (FLOAT)
  15. Airline code: The code with which each airlines is denoted (STRING)
  16. Airline: The airline name (STRING)
  17. Hotel:The hotel name (STRING)
  
 ###PROBLEM STATEMENT
The travel and tourism industry is one of the largest industries which contributes to trillions of dollars to the global economy. The analysis of travel data will be done in order to gain insights to data and help the travel and hospitality industry to offer various beneﬁts to the customers, increase marketing, strategic decisions etc. It will help the industry to better understand their target audience.In this project,we are analysing the travel data which includes data such as destination city, source city, mode of commute, the hotel they will be staying at, type of audience such as youth, children, adults etc. This project will prove helpful for both the hospitality industry and customers.

### Dataset Link

Download the dataset using the shell command wget and the URL, save them into the tmp directory. The URL for the dataset is
TravelData.csv : https://raw.githubusercontent.com/kavyaprasad/data/master/TravelData.csv

In [3]:
%sh
wget -P /tmp "https://raw.githubusercontent.com/kavyaprasad/data/master/TravelData.csv"

### Uploading the dataset into Databricks file system

Databricks file system is a distributed file system lying on top of Amazon S3. We will upload the data from the local file system into our DBFS. Below is a python script which copies the data from the local file system into the datasets folder of DBFS of your cluster.

The local files are referenced using `file:/` and DBFS files are referenced using `dbfs:/`

In [5]:
localTravelDataFilePath = "file:/tmp/TravelData.csv"
dbutils.fs.mkdirs("dbfs:/datasets")
dbutils.fs.cp(localTravelDataFilePath, "dbfs:/datasets/")
#Displaying the files present in the DBFS datasets folder of your cluser
display(dbutils.fs.ls("dbfs:/datasets"))

### Creating RDD for the Travel Data table
* Load the data into a RDD using the spark context method called TextFile.
* Print the formed RDD

In [7]:
travelRDD = sc.textFile("dbfs:/datasets/TravelData.csv")
for i in travelRDD.take(10): print(i)

###1) Top 10 destinations that people travel to
* Removed the header from the csv file, because we don't need the column names for the analysis.
* Split each record by taking the delimiter as comma because the data is comma separated in the CSV file.
* Created a key value pair where the key is the destination column and the corresponding value is 1.
* Apply a reduceByKey operation which counts the number of people going to the different destinations.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD

In [9]:
header= travelRDD.first()
travelFilterRDD = travelRDD.filter(lambda line:line!=header)
travelSplitRDD = travelFilterRDD.map(lambda row: row.split(","))
travelMapRDD = travelSplitRDD.map(lambda row: (row[2],1))
travelReducedRDD = travelMapRDD.reduceByKey(lambda acc,val: acc+val)
travelSwappedRDD=travelReducedRDD.map(lambda (a, b): (b, a))
travelDescendingRDD=travelSwappedRDD.sortByKey(0)
for i in travelDescendingRDD.take(20): print(i)  

###2) Top 10 destinations that people travel from

* Create a key value pair where the key is the origin column and the corresponding value is 1.
* Apply a reduceByKey operation which counts the number of people going from the different destinations.
* map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD

In [11]:
travelMapRDD1 = travelSplitRDD.map(lambda row: (row[1],1))
travelReducedRDD1 = travelMapRDD1.reduceByKey(lambda acc,val: acc+val)
travelSwappedRDD1=travelReducedRDD1.map(lambda (a, b): (b, a))
travelDescendingRDD1=travelSwappedRDD1.sortByKey(0)
for i in travelDescendingRDD1.take(20): print(i)

###3) Top 10 cities that generate high airline revenues for travel

* Filter the fourth column where the value is 1 which represents air mode of transport and take those rows and store it in another RDD.
* From the chosen rows, create a key-value pair where the key is the destination column and the corresponding value is 1.
* Apply a reduceByKey operation which counts the number of people going to different destinations.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD

In [13]:
new_travelSplitRDD=travelSplitRDD.filter(lambda x : x[3]=="1")
TravelMapRDD2=new_travelSplitRDD.map(lambda x : (x[2],1)).reduceByKey(lambda acc, val : acc+val)
travelSwappedRDD2=TravelMapRDD2.map(lambda (a, b): (b, a))
travelDescendingRDD2=travelSwappedRDD2.sortByKey(0)
for i in travelDescendingRDD2.take(20): print(i)

###4)Top 10 source-destination sites that people prefer travelling

* Create a key-value pair where the key is the source-destination column and the corresponding value is 1.
* Apply a reduceByKey operation which counts the number of people going to and trvelling from different destinations.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD

In [16]:
travelMapRDD3 = travelSplitRDD.map(lambda row: (row[0],1))
travelReducedRDD3 = travelMapRDD3.reduceByKey(lambda acc,val: acc+val)
travelSwappedRDD3=travelReducedRDD3.map(lambda (a, b): (b, a))
travelDescendingRDD3=travelSwappedRDD3.sortByKey(0)
for i in travelDescendingRDD3.take(20): print(i)

###5)Top 10 places from where people prefer travelling from their cars

* Filter the fourth column where the value is 2 which represents car mode of transport and take those rows and store it in another RDD.
* From the chosen rows, create a key-value pair where the key is the destination column and the corresponding value is 1.
* Apply a reduceByKey operation which counts the number of people going to different destinations.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD

In [18]:
new_travelSplitRDD4=travelSplitRDD.filter(lambda x : x[3]=="2")
TravelMapRDD4=new_travelSplitRDD4.map(lambda x : (x[1],1)).reduceByKey(lambda acc, val : acc+val)
travelSwappedRDD4=TravelMapRDD4.map(lambda (a, b): (b, a))
travelDescendingRDD4=travelSwappedRDD4.sortByKey(0)
for i in travelDescendingRDD4.take(20): print(i)

###6)Which of the combinations do people prefer the most (Air+Hotel or Hotel+Car orAir+Hotel+Car)
* Map through the RDD and create a key-value pair where key is the fourth column wherein the different combinations are present, filter the various combinations and take the required combinations that is 5,6 or 7. 
* Apply a reduceByKey operation which counts the number of people using these combinations.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD

In [20]:
TravelMapRDD5=travelSplitRDD.map(lambda x : (x[3],1)).reduceByKey(lambda acc, val : acc+val).filter(lambda x :x[0]== "5" or x[0]=="6" or x[0]== "7") 
travelSwappedRDD5=TravelMapRDD5.map(lambda (a, b): (b, a))
travelDescendingRDD5=travelSwappedRDD5.sortByKey(0)
for i in travelDescendingRDD5.take(20): print(i)

###7)Which of the hotels do people visit the most
* Map through the RDD and create a key-value pair where key is the eighteenth column and value is 1.
* Apply a reduceByKey operation which counts the the different hotels that people prefer going to.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method.
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD.

In [22]:
travelMapRDD6 = travelSplitRDD.map(lambda row: (row[16],1))
travelReducedRDD6 = travelMapRDD6.reduceByKey(lambda acc,val: acc+val)
travelSwappedRDD6=travelReducedRDD6.map(lambda (a, b): (b, a))
travelDescendingRDD6=travelSwappedRDD6.sortByKey(0)
for i in travelDescendingRDD6.take(20): print(i)

###8)Which airlines do people prefer travelling by
* Map through the RDD and create a key-value pair where key is the sixteenth column and value is 1.
* Apply a reduceByKey operation which counts the the different airlines that people prefer travelling in.
* Map through this RDD and swap the key-value pair, so that in the next step, the values can be counted using sortByKey method.
* Apply a sortByKey function on the swapped RDD, to list the RDD in descending order, enter 0 which represents false.
* Use a take action to print the created RDD.

In [24]:
travelMapRDD7 = travelSplitRDD.map(lambda row: (row[15],1))
travelReducedRDD7 = travelMapRDD7.reduceByKey(lambda acc,val: acc+val)
travelSwappedRDD7=travelReducedRDD7.map(lambda (a, b): (b, a))
travelDescendingRDD7=travelSwappedRDD7.sortByKey(0)
for i in travelDescendingRDD7.take(20): print(i)

### Distribution of Work within the team
 
 Nandini Goswami was responsible for 1st, 2nd and 3rd queries.
 
 Sarita Bhateja was responsible for 4th, 5th and 6th queries.
 
 Kavya Gururasad was responsible for 7th, 8th query and assembling the notebook.